In [3]:
import numpy as np
from scipy.stats.distributions import norm
import matplotlib.pylab as plt
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import sys
sys.path.append('../zdrojaky')
from tsplot import tsplot

# Téma 4: Modely MA a ARMA

## Moving average, MA(q) - modely klouzavých průměrů

## Smíšené modely ARMA(p, q)

### MLE odhad AR(1) procesu

Uvažujme stacionární gaussovský AR(1) proces

$$
X_t = c + \phi X_{t-1} + \varepsilon_t, \qquad \varepsilon_t \sim iid \mathcal{N}(0, \sigma^2), |\phi|<1 \text{(proč?)}
$$

Vektor neznámých parametrů tedy je $\theta = [c, \phi, \sigma^2]^\intercal$.

Z minulé přednášky už víme, že

$$
\mathbb{E}[X_1] = \mu = \frac{c}{1-\phi}, \qquad \operatorname{var} X_1 = \frac{\sigma^2}{1-\phi^2},
$$

a jelikož šum je normální (gaussovský), platí

$$
X_1 \sim \mathcal{N}(\mu, \operatorname{var} X_1) = \mathcal{N}\left( \frac{c}{1-\phi}, \frac{\sigma^2}{1-\phi^2}\right).
$$

Připomeňme, že hustota pravděpodobnosti takového normálního rozdělení je

$$
f(x_1; \theta) = f(x_1; c, \phi, \sigma^2) = \frac{1}{\sqrt{2\pi} \sqrt{\sigma^2/(1 - \phi^2)}}
\exp
\left[
-\frac{1}{2} \frac{[x_1 - [c/(1-\phi)]^2}{\sigma^2/(1-\phi^2)}
\right].
$$

Podívejme se nyní na druhé pozorování $X_2$, které je v autoregresním modelu podmíněné $X_1$,

$$
X_2 = c + \phi X_1 + \varepsilon_2 \qquad \Rightarrow X_2|X_1=x_1 \sim \mathcal{N}\left((c+\phi x_1), \sigma^2\right),
$$

neboť zde je $X_1$ bráno jako deterministická konstanta $x_1$. Tedy ke konstantě $(c+\phi x_1)$ přičítáme šum z $\mathcal{N}(0, \sigma^2)$ a

$$
f(x_2|x_1;\theta) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp 
\left[
-\frac{1}{2} \frac{(x_2 - c - \phi x_1)^2}{\sigma^2}
\right].
$$

Podobně budeme mít $(X_3|X_2, X_1) = (X_3|X_2)$ atd., neboť do modelu v čase $t$ vstupuje na pravé straně jen $X_{t-1}$. Tedy hustota v čase $t$ je

$$
f(x_t|x_{t-1}, x_{t-2}, \ldots, x_1; \theta) = f(x_t|x_{t-1}; \theta)
$$

a věrohodnostní funkce ("pravděpodobnost") všech pozorování do času $T$

$$
f(x_T, x_{T-1},\ldots, x_1;\theta) = f(x_1;\theta) \prod_{t=2}^{T} f(x_t|x_{t-1}; \theta), \qquad\text{kde}\ \theta = [c, \phi, \sigma^2]^\intercal.
$$

Logaritmus věrohodnosti - log-likelihood - je tedy

$$
\begin{aligned}
\mathcal{L}(\theta) &= \log f(x_1;\theta) + \sum_{t=2}^{T} \log f(x_t|x_{t-1}; \theta) \\
&= \underbrace{
-\frac{1}{2} \log (2\pi) - \frac{1}{2} \log\left(\frac{\sigma^2}{1-\phi^2}\right)
- \frac{\left( x_1 - \frac{c}{1-\phi}\right)^2}{2 \frac{\sigma^2}{1-\phi^2}}
}_{\log f(x_1;\theta)}
\underbrace{
-\frac{T-1}{2} \log(2\pi) - \frac{T-1}{2}\log(\sigma^2) - \sum_{t=1}^{T} \frac{\varepsilon_t^2}{2\sigma^2}
}_{\log\prod f(x_t|x_{t-1};\theta)},
\end{aligned}
$$

kde $\varepsilon_t = x_t - c - \phi x_{t-1}$. Ta se nám budou hodit u odhadu MA části.

Nyní bychom položili log-likelihood rovný nule a hledali maximum přes parametry $\theta = [c, \phi, \sigma^2]^\intercal$, což je úkol iterativní a pro počítač :-)

### Conditional MLE - metoda podmíněné maximální věrohodnosti pro AR(1)

Alternativně k výše uvedené plné věrohodnosti lze uvažovat hodnotu $y_1$ deterministickou a tedy $f(y_1) = 1$. Potom maximalizujeme log-likelihood podmíněný prvním pozorováním, tedy jen tu součtovou část $\sum \log f(x_t|x_{t-1}; \theta)$,

$$
\mathcal{L}^*(\theta) = -\frac{T-1}{2} \log(2\pi) - \frac{T-1}{2}\log(\sigma^2) - \sum_{t=1}^{T} \frac{\varepsilon_t^2}{2\sigma^2}.
$$

Dá se ukázat, že toto odpovídá odhadu pomocí nejmenších čtverců.

## MLE pro AR(p)

Mějme nyní autoregresní model řádu $p$, tedy

$$
X_t = c + \phi_1 X_{t-1} + \phi_2 X_{t-2} + \ldots + \phi_p X_{t-p} + \varepsilon_t, \qquad \varepsilon_t \sim iid \mathcal{N}(0, \sigma^2).
$$

Nyní $\theta = [c, \phi_1, \ldots, \phi_p, \sigma^2]^\intercal$. Kdybychom postupovali jako u AR(1), ukázalo by se, že podmíněná maximální věrohodnost bude pracovat s log-likelihoodem

$$
\mathcal{L}^*(\theta) = -\frac{T-p}{2} \log(2\pi) - \frac{T-p}{2}\log(\sigma^2) - \sum_{t=p1}^{T} \frac{\varepsilon_t^2}{2\sigma^2}.
$$